In [2]:
# Import our dependencies
import pandas as pd
import numpy as np
from pathlib import Path

# Send output to database
from sqlalchemy import create_engine
# Importing config file for pulling from database
from config import config

In [3]:
# Read a starting database
file_path = "Resources/14100328.csv"
data = pd.read_csv(file_path, error_bad_lines=False)
data.sample(n=3)

/Users/oi/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,REF_DATE,GEO,DGUID,National Occupational Classification,Job vacancy characteristics,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
9069575,2016-10,Nunavut,2016A000262,Corporate sales managers [060],Seasonal,Average offered hourly wage,Dollars,81,units,0,v105690177,14.63.33.5,NaN,..,NaN,NaN,2
16933210,2018-07,Northwest Territories,2016A000261,Archivists [5113],8 years or more,Proportion of job vacancies,Percentage,242,units,0,v105636683,13.433.22.2,NaN,..,NaN,NaN,1
15115498,2018-04,Nova Scotia,2016A000212,Boat and cable ferry operators and related occ...,"College, CEGEP and other non-university certif...",Average offered hourly wage,Dollars,81,units,0,v104843325,4.591.9.5,NaN,..,NaN,NaN,2


<h1><span style="color:red"> I. Removing Redundant Data</span></h1>

### Filter data quality: A - excellent, B - very good, C - good, acceptable - D
### Making Ref Date into actual Date

In [4]:
df = data.copy()
df = df.loc[(df['STATUS'] == 'A')|(df['STATUS'] == 'B')|(df['STATUS'] == 'C')|(df['STATUS'] == 'D')]

#appending date to allow for date time data type
df['REF_DATE'] = df['REF_DATE'] + '-01'

df['REF_DATE'].unique()

array(['2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01',
       '2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01',
       '2017-01-01', '2017-04-01', '2017-07-01', '2017-10-01',
       '2018-01-01', '2018-04-01', '2018-07-01', '2018-10-01',
       '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01',
       '2020-01-01', '2020-10-01', '2021-01-01', '2021-04-01'],
      dtype=object)

## Filtering for Vacancies and Average Wage only

In [5]:
#Filtering for Vacancies and Average Wage
df = df.loc[(df['Statistics'] =='Job vacancies') | (df['Statistics'] == 'Average offered hourly wage')]

In [6]:
#splitting NOC Description and Code
df[['NOCdesc','NOC']] = df['National Occupational Classification'].str.split("[",expand=True)

#removing junk from column
df[['NOC','junk']] = df['NOC'].str.split("]", expand=True)

#dropping columns not required, renaming and reordering columns
df = df.drop(columns=['DGUID','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','STATUS','SYMBOL','COORDINATE',
                 'TERMINATED','DECIMALS','junk','National Occupational Classification'])

df = df.rename(columns={"Job vacancy characteristics" :"JOB_CHAR"})
df = df[['REF_DATE','GEO','NOC','NOCdesc','JOB_CHAR','Statistics','VALUE']]

df.head()

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,None,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.15
11,2015-01-01,Canada,None,"Total, all occupations","Minimum level of education required, all levels",Average offered hourly wage,19.15
38,2015-01-01,Canada,None,"Total, all occupations","Certification requirement, all types",Average offered hourly wage,19.15
47,2015-01-01,Canada,None,"Total, all occupations","Minimum experience level sought, all levels",Average offered hourly wage,19.15
65,2015-01-01,Canada,None,"Total, all occupations","Duration of job vacancy, all durations",Average offered hourly wage,19.15


In [7]:
#stripping trailing spaces
df["NOCdesc"] = df["NOCdesc"].str.rstrip()

In [8]:
df.isnull().sum()

REF_DATE          0
GEO               0
NOC           25599
NOCdesc           0
JOB_CHAR          0
Statistics        0
VALUE             0
dtype: int64

In [9]:
#checking NOC Descriptions of null NOCs
nullNoc = df[df['NOC'].isna()]
nullNoc['NOCdesc'].unique()

array(['Total, all occupations', 'Unclassified occupations'], dtype=object)

In [10]:
#replacing Null NOCs for Total,all occupations and Unclassified Occupations
temp = df['NOC'].isna()
df.loc[temp, 'NOC'] = np.where(df.loc[temp, 'NOCdesc'].eq('Total, all occupations'), '101', 'X')

In [11]:
#checking outcome
df[df['NOCdesc'] == 'Total, all occupations']
#df[df['NOCdesc'] == 'Unclassified occupations']

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,101,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.15
11,2015-01-01,Canada,101,"Total, all occupations","Minimum level of education required, all levels",Average offered hourly wage,19.15
38,2015-01-01,Canada,101,"Total, all occupations","Certification requirement, all types",Average offered hourly wage,19.15
47,2015-01-01,Canada,101,"Total, all occupations","Minimum experience level sought, all levels",Average offered hourly wage,19.15
65,2015-01-01,Canada,101,"Total, all occupations","Duration of job vacancy, all durations",Average offered hourly wage,19.15
...,...,...,...,...,...,...,...
27254446,2021-04-01,Nunavut,101,"Total, all occupations",Seasonal,Average offered hourly wage,32.95
27254447,2021-04-01,Nunavut,101,"Total, all occupations","Recruitment strategies, all types",Job vacancies,615.00
27254449,2021-04-01,Nunavut,101,"Total, all occupations","Personal contacts, referrals, informal networks",Job vacancies,445.00
27254455,2021-04-01,Nunavut,101,"Total, all occupations",Online job boards,Job vacancies,475.00



### Creating input table for Machine Learning, filtered on Broad NOC Code, Full/Part Time only

In [12]:
df_vac = df.copy()

# Filter job vacancies
df_vac = df_vac.loc[(df_vac['Statistics'] =='Job vacancies')]

# Filter Job vacancy characteristics
df_vac = df_vac.loc[(df_vac['JOB_CHAR'] == 'Full-time') | (df_vac['JOB_CHAR'] == 'Part-time') | (df_vac['JOB_CHAR'] == 'Type of work, all types')]

#drop column not required
df_vac = df_vac.drop(columns=['Statistics'])

df_vac

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,VALUE
408280,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60505.0
408283,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45385.0
653248,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14230.0
653251,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10375.0
1143184,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",451925.0
...,...,...,...,...,...,...
27263184,2021-04-01,Nunavut,1,"Business, finance and administration occupations",Full-time,70.0
27309524,2021-04-01,Nunavut,7,"Trades, transport and equipment operators and ...","Type of work, all types",160.0
27309527,2021-04-01,Nunavut,7,"Trades, transport and equipment operators and ...",Full-time,160.0
27314606,2021-04-01,Nunavut,73,Maintenance and equipment operation trades,"Type of work, all types",65.0


In [13]:
# Machine Learning only needs Broad NOC
inc_list = ['101','0','1', '2', '3', '4', '5', '6', '7', '8', '9']

#extract into new dataframe
df_ml = df_vac[df_vac.NOC.isin(inc_list)]

#adding null column to hold predicted vacancies
df_ml["Predicted_Vacancies"] = ''

df_ml

/Users/oi/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,VALUE,Predicted_Vacancies
408280,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60505.0,
408283,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45385.0,
653248,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14230.0,
653251,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10375.0,
1143184,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",451925.0,
...,...,...,...,...,...,...,...
27254351,2021-04-01,Nunavut,101,"Total, all occupations",Full-time,515.0,
27263181,2021-04-01,Nunavut,1,"Business, finance and administration occupations","Type of work, all types",75.0,
27263184,2021-04-01,Nunavut,1,"Business, finance and administration occupations",Full-time,70.0,
27309524,2021-04-01,Nunavut,7,"Trades, transport and equipment operators and ...","Type of work, all types",160.0,


In [14]:
df_ml=df_ml.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", "JOB_CHAR": "job_char", "NOCdesc": "noc_desc", "NOC": "noc_code", "VALUE":"total_vacancies", "Predicted_Vacancies": "predicted_vacancies"})
df_ml

,ref_date,geo,noc_code,noc_desc,job_char,total_vacancies,predicted_vacancies
408280,2015-01-01,Quebec,101,"Total, all occupations","Type of work, all types",60505.0,
408283,2015-01-01,Quebec,101,"Total, all occupations",Full-time,45385.0,
653248,2015-01-01,Saskatchewan,101,"Total, all occupations","Type of work, all types",14230.0,
653251,2015-01-01,Saskatchewan,101,"Total, all occupations",Full-time,10375.0,
1143184,2015-04-01,Canada,101,"Total, all occupations","Type of work, all types",451925.0,
...,...,...,...,...,...,...,...
27254351,2021-04-01,Nunavut,101,"Total, all occupations",Full-time,515.0,
27263181,2021-04-01,Nunavut,1,"Business, finance and administration occupations","Type of work, all types",75.0,
27263184,2021-04-01,Nunavut,1,"Business, finance and administration occupations",Full-time,70.0,
27309524,2021-04-01,Nunavut,7,"Trades, transport and equipment operators and ...","Type of work, all types",160.0,


In [15]:
df_ml.to_csv('Resources/MachineLearning.csv', index=True, index_label="idML")

In [16]:
engine = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:5432/{config['dbname']}")
conn=engine.connect()
df_ml.to_sql(name="machinelearning", con=conn, index=True, index_label="idML", if_exists="replace")
conn.close()

## Creating Vacancies Table data

In [17]:
#adding YEAR and Quarter for Vacancies table

#splitting date to year and month
df_vac[['Year','Quarter','Date']] = df_vac['REF_DATE'].str.split("-",expand=True)                  

#converting Month to Quarter
df_vac['Quarter'] = df_vac['Quarter'].replace(['01','04','07','10'],['1','2','3','4'])

#reordeing columns
df_vac = df_vac[['REF_DATE','Year', 'Quarter', 'GEO','NOC','JOB_CHAR','VALUE']]
df_vac

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
408280,2015-01-01,2015,1,Quebec,101,"Type of work, all types",60505.0
408283,2015-01-01,2015,1,Quebec,101,Full-time,45385.0
653248,2015-01-01,2015,1,Saskatchewan,101,"Type of work, all types",14230.0
653251,2015-01-01,2015,1,Saskatchewan,101,Full-time,10375.0
1143184,2015-04-01,2015,2,Canada,101,"Type of work, all types",451925.0
...,...,...,...,...,...,...,...
27263184,2021-04-01,2021,2,Nunavut,1,Full-time,70.0
27309524,2021-04-01,2021,2,Nunavut,7,"Type of work, all types",160.0
27309527,2021-04-01,2021,2,Nunavut,7,Full-time,160.0
27314606,2021-04-01,2021,2,Nunavut,73,"Type of work, all types",65.0


In [18]:
df_vac=df_vac.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", 
                              "JOB_CHAR": "job_char", "NOC": "noc_code", "VALUE":"total_vacancies"})

In [19]:
#exporting to csv
df_vac.to_csv('Resources/AllVacancies.csv', index=True, index_label="idVac")

In [20]:
engine2 = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:5432/{config['dbname']}")
conn=engine2.connect()
df_vac.to_sql(name="vacancies", con=conn, index=True, index_label="idVac", if_exists="replace")
conn.close()

## NOC Table

In [21]:
## creating NOC dataframe
NOC = df[['NOC','NOCdesc']]

#dropping duplicate rows
NOC = NOC.drop_duplicates()

NOC

,NOC,NOCdesc
2,101,"Total, all occupations"
8616,1,"Business, finance and administration occupations"
44134,6,Sales and service occupations
45432,63,Service supervisors and specialized service oc...
48500,65,Service representatives and other customer and...
...,...,...
14866806,7234,Boilermakers
15972060,5135,Actors and comedians
15999422,826,Fishing vessel masters and fishermen/women
22720601,844,Other workers in fishing and trapping and hunt...


In [22]:
engine3 = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:5432/{config['dbname']}")
conn=engine3.connect()
NOC.to_sql(name="noc", con=conn, index=False, if_exists="replace")
conn.close()

In [23]:
#exporting to csv
NOC.to_csv('Resources/NOCtable.csv', index=False)

## Creating Data for Average Wage Table

In [24]:
avg_wage = df.copy()

In [25]:
# Filter job vacancies
avg_wage = avg_wage.loc[(avg_wage['Statistics'] =='Average offered hourly wage')]

# Filter number of Job vacancies
avg_wage = avg_wage.loc[(avg_wage['JOB_CHAR'] == 'Full-time') | (avg_wage['JOB_CHAR'] == 'Part-time') | (avg_wage['JOB_CHAR'] == 'Type of work, all types')]

avg_wage

,REF_DATE,GEO,NOC,NOCdesc,JOB_CHAR,Statistics,VALUE
2,2015-01-01,Canada,101,"Total, all occupations","Type of work, all types",Average offered hourly wage,19.15
8616,2015-01-01,Canada,1,"Business, finance and administration occupations","Type of work, all types",Average offered hourly wage,21.35
44134,2015-01-01,Canada,6,Sales and service occupations,"Type of work, all types",Average offered hourly wage,13.00
45432,2015-01-01,Canada,63,Service supervisors and specialized service oc...,"Type of work, all types",Average offered hourly wage,13.75
48500,2015-01-01,Canada,65,Service representatives and other customer and...,"Type of work, all types",Average offered hourly wage,12.40
...,...,...,...,...,...,...,...
27315458,2021-04-01,Nunavut,731,Machinery and transportation equipment mechani...,Full-time,Average offered hourly wage,37.30
27315576,2021-04-01,Nunavut,7311,Construction millwrights and industrial mechanics,"Type of work, all types",Average offered hourly wage,34.70
27315579,2021-04-01,Nunavut,7311,Construction millwrights and industrial mechanics,Full-time,Average offered hourly wage,34.70
27319811,2021-04-01,Nunavut,75,Transport and heavy equipment operation and re...,"Type of work, all types",Average offered hourly wage,22.45


In [26]:
#splitting date to year and month
avg_wage[['Year','Quarter','Date']] = avg_wage['REF_DATE'].str.split("-",expand=True)                  

#converting Month to Quarter
avg_wage['Quarter'] = avg_wage['Quarter'].replace(['01','04','07','10'],['1','2','3','4'])

#drop column not required
avg_wage = avg_wage.drop(columns=['Date'])
avg_wage = avg_wage.drop(columns=['Statistics'])

#reordeing columns
avg_wage = avg_wage[['REF_DATE','Year', 'Quarter', 'GEO','NOC','JOB_CHAR','VALUE']]

avg_wage

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.15
8616,2015-01-01,2015,1,Canada,1,"Type of work, all types",21.35
44134,2015-01-01,2015,1,Canada,6,"Type of work, all types",13.00
45432,2015-01-01,2015,1,Canada,63,"Type of work, all types",13.75
48500,2015-01-01,2015,1,Canada,65,"Type of work, all types",12.40
...,...,...,...,...,...,...,...
27315458,2021-04-01,2021,2,Nunavut,731,Full-time,37.30
27315576,2021-04-01,2021,2,Nunavut,7311,"Type of work, all types",34.70
27315579,2021-04-01,2021,2,Nunavut,7311,Full-time,34.70
27319811,2021-04-01,2021,2,Nunavut,75,"Type of work, all types",22.45


In [27]:
avg_wage.drop_duplicates(keep='first', ignore_index=False, inplace=True)
avg_wage

,REF_DATE,Year,Quarter,GEO,NOC,JOB_CHAR,VALUE
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.15
8616,2015-01-01,2015,1,Canada,1,"Type of work, all types",21.35
44134,2015-01-01,2015,1,Canada,6,"Type of work, all types",13.00
45432,2015-01-01,2015,1,Canada,63,"Type of work, all types",13.75
48500,2015-01-01,2015,1,Canada,65,"Type of work, all types",12.40
...,...,...,...,...,...,...,...
27315458,2021-04-01,2021,2,Nunavut,731,Full-time,37.30
27315576,2021-04-01,2021,2,Nunavut,7311,"Type of work, all types",34.70
27315579,2021-04-01,2021,2,Nunavut,7311,Full-time,34.70
27319811,2021-04-01,2021,2,Nunavut,75,"Type of work, all types",22.45


In [28]:
avg_wage=avg_wage.rename(columns={"REF_DATE": "ref_date", "GEO": "geo", 
                              "JOB_CHAR": "job_char", "NOC": "noc_code", "VALUE":"avg_wage"})
avg_wage

,ref_date,Year,Quarter,geo,noc_code,job_char,avg_wage
2,2015-01-01,2015,1,Canada,101,"Type of work, all types",19.15
8616,2015-01-01,2015,1,Canada,1,"Type of work, all types",21.35
44134,2015-01-01,2015,1,Canada,6,"Type of work, all types",13.00
45432,2015-01-01,2015,1,Canada,63,"Type of work, all types",13.75
48500,2015-01-01,2015,1,Canada,65,"Type of work, all types",12.40
...,...,...,...,...,...,...,...
27315458,2021-04-01,2021,2,Nunavut,731,Full-time,37.30
27315576,2021-04-01,2021,2,Nunavut,7311,"Type of work, all types",34.70
27315579,2021-04-01,2021,2,Nunavut,7311,Full-time,34.70
27319811,2021-04-01,2021,2,Nunavut,75,"Type of work, all types",22.45


In [29]:
#exporting to csv
avg_wage.to_csv('Resources/AvgWagetable.csv', index = False)

In [30]:
engine4 = create_engine(f"postgresql://{config['user']}:{config['password']}@{config['host']}:5432/{config['dbname']}")
conn=engine4.connect()
avg_wage.to_sql(name="averagewage", con=conn, index=True, index_label="idW", if_exists="replace")
conn.close()